# 💭 Project 5: The Infinite Companion

**Objective:** Build a chatbot that remembers facts about you across sessions.

## 📖 The Memory Problem

LLMs have a context window limit (e.g., 8K, 32K tokens). For long conversations:
- You can't fit entire history
- Information gets lost
- No persistence across sessions

## 🎯 Solution: Memory Systems

1. **SummaryMemory**: Compress old messages into summaries
2. **EntityMemory**: Extract and store key facts
3. **Sliding Window**: Keep only recent messages
4. **Semantic Retrieval**: Fetch relevant past context

In [ ]:
import os
import json
import sqlite3
from datetime import datetime
from typing import List, Dict, Any
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

print("✅ Setup complete!")

## Task 1: Implement Summary Memory

In [ ]:
class SummaryMemory:
    """
    Maintains conversation summary when messages exceed limit.
    """
    def __init__(self, max_messages: int = 10):
        self.max_messages = max_messages
        self.summary = ""
        self.recent_messages = []
    
    def add_message(self, role: str, content: str):
        """Add a message and summarize if needed."""
        self.recent_messages.append({"role": role, "content": content})
        
        if len(self.recent_messages) > self.max_messages:
            self._summarize_and_compress()
    
    def _summarize_and_compress(self):
        """Summarize older messages and keep only recent ones."""
        # Take first half for summarization
        to_summarize = self.recent_messages[:self.max_messages // 2]
        
        # Create summary prompt
        messages_text = "\n".join([
            f"{msg['role']}: {msg['content']}" for msg in to_summarize
        ])
        
        prompt = f"""Summarize this conversation, preserving key facts and context:

{messages_text}

Previous summary: {self.summary if self.summary else 'None'}

Provide a concise summary:"""
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        
        self.summary = response.choices[0].message.content
        self.recent_messages = self.recent_messages[self.max_messages // 2:]
        
        print(f"📝 Summarized {len(to_summarize)} messages")
    
    def get_context(self) -> List[Dict[str, str]]:
        """Get formatted context for LLM."""
        context = []
        
        if self.summary:
            context.append({
                "role": "system",
                "content": f"Conversation summary: {self.summary}"
            })
        
        context.extend(self.recent_messages)
        return context

# Test
memory = SummaryMemory(max_messages=4)
print("✅ SummaryMemory initialized")

## Task 2: Implement Entity Memory

In [ ]:
class EntityMemory:
    """
    Extracts and stores facts about entities (users, topics, etc.).
    """
    def __init__(self, db_path: str = "entity_memory.db"):
        self.db_path = db_path
        self._init_db()
    
    def _init_db(self):
        """Initialize SQLite database."""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS entities (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                entity_type TEXT,
                entity_name TEXT,
                fact TEXT,
                timestamp TEXT
            )
        ''')
        conn.commit()
        conn.close()
    
    def extract_and_store(self, message: str):
        """Extract entities and facts from message."""
        prompt = f"""Extract key facts from this message. Format as JSON list:
[{{"type": "person/preference/fact", "entity": "name", "fact": "description"}}]

Message: {message}

Only extract significant, memorable facts. Return empty list if none."""
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        
        try:
            entities = json.loads(response.choices[0].message.content)
            self._store_entities(entities)
            return len(entities)
        except:
            return 0
    
    def _store_entities(self, entities: List[Dict]):
        """Store entities in database."""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        for entity in entities:
            cursor.execute('''
                INSERT INTO entities (entity_type, entity_name, fact, timestamp)
                VALUES (?, ?, ?, ?)
            ''', (
                entity.get('type', 'unknown'),
                entity.get('entity', 'unknown'),
                entity.get('fact', ''),
                datetime.now().isoformat()
            ))
        
        conn.commit()
        conn.close()
    
    def get_all_facts(self) -> str:
        """Get all stored facts as text."""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('SELECT entity_type, entity_name, fact FROM entities')
        results = cursor.fetchall()
        conn.close()
        
        if not results:
            return "No facts stored yet."
        
        facts = [f"- {entity_name} ({entity_type}): {fact}" 
                for entity_type, entity_name, fact in results]
        return "\n".join(facts)

# Test
entity_memory = EntityMemory()
print("✅ EntityMemory initialized")

## Task 3: Build Chatbot with Memory

In [ ]:
class MemoryChatbot:
    """
    Chatbot with both summary and entity memory.
    """
    def __init__(self):
        self.summary_memory = SummaryMemory(max_messages=8)
        self.entity_memory = EntityMemory()
        self.system_prompt = """You are a friendly AI assistant with perfect memory.
You remember facts about the user and use them in conversation naturally.
Be warm, engaging, and show that you remember previous conversations."""
    
    def chat(self, user_message: str) -> str:
        """Process a message and return response."""
        # Extract and store entities from user message
        facts_extracted = self.entity_memory.extract_and_store(user_message)
        if facts_extracted > 0:
            print(f"💾 Stored {facts_extracted} new facts")
        
        # Add to conversation memory
        self.summary_memory.add_message("user", user_message)
        
        # Build context with stored facts
        all_facts = self.entity_memory.get_all_facts()
        
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "system", "content": f"Facts you know:\n{all_facts}"},
        ]
        messages.extend(self.summary_memory.get_context())
        
        # Get response
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0.7
        )
        
        assistant_message = response.choices[0].message.content
        self.summary_memory.add_message("assistant", assistant_message)
        
        return assistant_message

# Initialize chatbot
bot = MemoryChatbot()
print("🤖 Memory Chatbot ready!")

## 🧪 Test the Chatbot

In [ ]:
# Conversation 1: Share information
print("User: Hi! My name is Alex and I love Python programming.")
response = bot.chat("Hi! My name is Alex and I love Python programming.")
print(f"Bot: {response}\n")

print("User: I'm working on an AI agent project right now.")
response = bot.chat("I'm working on an AI agent project right now.")
print(f"Bot: {response}\n")

print("User: My favorite food is pizza.")
response = bot.chat("My favorite food is pizza.")
print(f"Bot: {response}\n")

In [ ]:
# Conversation 2: Test memory recall (simulate new session)
bot2 = MemoryChatbot()  # New instance, but same database

print("User: What do you know about me?")
response = bot2.chat("What do you know about me?")
print(f"Bot: {response}\n")

print("User: What food do I like?")
response = bot2.chat("What food do I like?")
print(f"Bot: {response}")

## 🎓 Key Takeaways

### Memory Strategies:

1. **Summary Memory**: Compress history to fit context
2. **Entity Memory**: Extract and persist key facts
3. **Sliding Window**: Keep recent, summarize old
4. **Semantic Search**: Retrieve relevant past context (Phase 2!)

### Production Tips:

- Use vector DBs for semantic memory search
- Implement memory decay (older facts less relevant)
- Add user privacy controls
- Optimize summarization frequency

### Next: Planning!

Module 6 adds hierarchical planning for complex multi-step tasks.